In [ ]:
import os
import pathlib

# URL del repository pubblico
REPO_URL = 'https://github.com/ev91/MachineInnovators_Inc_ProAI.git'
repo_dir = pathlib.Path('/content/MachineInnovators_Inc_ProAI')

if not repo_dir.exists():
    print(f"[*] Clonazione di {REPO_URL}...")
    !git clone $REPO_URL
    print(f"[✓] Repository clonato in {repo_dir}")
else:
    print(f"[*] Repository già presente in {repo_dir}")

os.chdir(repo_dir)
print(f"[✓] Working directory: {repo_dir}")

## 2) Installa le dipendenze

In [ ]:
import sys
import os
import pathlib

project_root = pathlib.Path.cwd()

# Verifica che il repo sia nella working directory
if not (project_root / 'src').exists():
    alt = pathlib.Path('/content/MachineInnovators_Inc_ProAI')
    if alt.exists():
        project_root = alt
        os.chdir(project_root)
    else:
        raise RuntimeError('Repository non trovato: esegui la cella precedente per clonare')

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"[*] Installazione dipendenze da {project_root / 'requirements.txt'}...")
!pip install -q -r requirements.txt
print(f"[✓] Dipendenze installate")

# Configurazione MLflow
os.environ['MLFLOW_TRACKING_URI'] = f'file://{project_root / "mlruns"}'
os.environ['REGISTERED_MODEL_NAME'] = 'Sentiment'

print(f"[✓] MLflow tracking URI: {os.environ['MLFLOW_TRACKING_URI']}")
print(f"[✓] Model name: {os.environ['REGISTERED_MODEL_NAME']}")

## 3) Demo: Inferenza con il modello HuggingFace

Usiamo il modello pre-addestrato `cardiffnlp/twitter-roberta-base-sentiment-latest` per fare una predizione rapida.

In [ ]:
from src.serving.load_model import predict_fn

# Test su diversi testi
test_texts = [
    "I absolutely love this product!",
    "This is just okay, nothing special.",
    "Terrible experience, would not recommend.",
]

print("[*] Predizioni usando il modello HuggingFace:")
print("-" * 60)

for text in test_texts:
    label, score = predict_fn(text)
    print(f"Text: {text[:50]:50s} | Label: {label:8s} | Score: {score:.4f}")

print("[✓] Inferenza completata")

## 4) Addestra una nuova versione del modello

Questo step registra una nuova versione del modello nel MLflow Model Registry (file-based).

In [ ]:
print("[*] Avvio training...")
!python -m src.models.train_roberta --experiment sentiment_colab
print("[✓] Training completato")

## 5) Recupera l'ultima versione registrata dal Model Registry

In [ ]:
from mlflow.tracking import MlflowClient

model_name = os.environ.get('REGISTERED_MODEL_NAME', 'Sentiment')
tracking_uri = os.environ['MLFLOW_TRACKING_URI']

client = MlflowClient(tracking_uri=tracking_uri)

# Recupera tutte le versioni
all_versions = client.search_model_versions(f"name='{model_name}'")

if not all_versions:
    raise RuntimeError(f"Nessuna versione trovata per il modello '{model_name}'")

# Prendi la versione più recente
latest = sorted(all_versions, key=lambda v: int(v.version))[-1]
new_model_uri = f"models:/{model_name}/{latest.version}"

print(f"[✓] Ultime versioni registrate:")
for v in sorted(all_versions[-3:], key=lambda x: int(x.version), reverse=True):
    print(f"  - Version {v.version}: stage={v.current_stage}")

print(f"[✓] Usando versione: {new_model_uri}")

## 6) Valuta e promuovi a Production

Usiamo `data/holdout.csv` come set di valutazione. Se il nuovo modello è >= della versione in Production (o se nessuno è in Production), viene promosso.

In [ ]:
print("[*] Valutazione del modello e promozione a Production...")
!python -m src.models.evaluate --new_model_uri $new_model_uri --eval_csv data/holdout.csv --min_improvement 0.0
print("[✓] Valutazione e promozione completate")

## 7) Carica il modello in Production dal registry

In [ ]:
from src.utils.mlflow_utils import get_production_model_uri

prod_uri = get_production_model_uri(model_name)

if prod_uri:
    os.environ['MODEL_URI'] = prod_uri
    print(f"[✓] Production URI trovata: {prod_uri}")
else:
    os.environ['MODEL_URI'] = ''
    print(f"[!] Production URI non trovata, fallback a HuggingFace")

## 8) Inferenza usando il modello in Production

Ora usiamo il modello registrato in Production (se disponibile) altrimenti fallback su HuggingFace.

In [ ]:
# Reload predict_fn per utilizzare il nuovo MODEL_URI
import importlib
import src.serving.load_model
importlib.reload(src.serving.load_model)
from src.serving.load_model import predict_fn

print(f"[*] Predizioni usando MODEL_URI={os.environ.get('MODEL_URI', 'None (fallback HF)')}")
print("-" * 60)

test_texts = [
    "This is amazing! Best service ever!",
    "Not the best, could be better.",
    "Absolutely horrible, never again!",
]

for text in test_texts:
    label, score = predict_fn(text)
    print(f"Text: {text[:50]:50s} | Label: {label:8s} | Score: {score:.4f}")

print("[✓] Inferenza completata")